In [2]:
#导入需要的模块
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from collections import Counter

#定义一些超参数
BATCHSIZE = 100
DOWNLOAD_MNIST = False
EPOCHES=20
LR = 0.001

定义相关模型结构：CNNNet, LeNet, VGG

In [3]:
class CNNNet(nn.Module):
    def __init__(self):
        super(CNNNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels= 16, kernel_size=5, stride=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=36, kernel_size=3,stride=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        #36 x 6 x 6
        self.fc1 = nn.Linear(1296,128)
        self.fc2 = nn.Linear(128,10)
    def forward(self,x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
#         print("x's shape is {}".format(x.shape))
        x = x.view(-1,36*6*6)
        x = F.relu(self.fc2(F.relu(self.fc1(x))))
        return x

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 36, 5)
        self.pool2 = nn.MaxPool2d(2,2)
        self.aap = nn.AdaptiveAvgPool2d(1)#output 样本数 x 36 x 1 x 1
        self.fc3 = nn.Linear(36, 10)
        
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
#         print("brfore aap: {}".format(x.shape))
        x = self.aap(x)
#         print("after aap: {}".format(x.shape))
        x = x.view(x.shape[0], -1)
#         print("after view, shape :{}".format(x.shape))
#         print("after view, size():{}".format(x.size()))
        x = self.fc3(x)
        return x

In [5]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [6]:
cfg = {
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512,10)
    def forward(self,x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out
    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                          nn.BatchNorm2d(x),
                          nn.ReLU(inplace=True)]
                #nn.ReLu在构建网络结构时加入，inplace = true可节省空间，F.relu()在forward函数中用
                in_channels = x
            layers += [nn.AvgPool2d(kernel_size = 1,stride = 1)]
        return nn.Sequential(*layers)
        

#导入数据，若已下载本地，设download=False

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#data
print('==>Preparing data..')
#transform对数据源进行预处理
transform_train = transforms.Compose([
    #随机裁剪图片
    transforms.RandomCrop(32, padding=4),
    #图像随机水平翻转
    transforms.RandomHorizontalFlip(),
    #将一个取值范围是[0,255]的PIL.Image转换成Tensor
    transforms.ToTensor(),
    #标准化，减均值，除以标准差
    transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root= '../../pytorch-data/data', train=True,download=False,transform=transform_train)
#num_workers一般经验值为自己电脑的cpu核心数
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root = '../../pytorch-data/data', train=False,download=False,transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCHSIZE, shuffle=False, num_workers=2)

classes = ('plane','car','bird','cat','deer','frog','horse','ship','truck')

#Model
print('==> Building model..')
net1 = CNNNet()
net2 = Net()
net3 = LeNet()
net4 = VGG('VGG16')



==>Preparing data..
==> Building model..


In [8]:
#将三个网络模型放在一个列表里,并将模型加载到指定设备上
mlps = [net1.to(device), net2.to(device), net3.to(device)]
optimizer = torch.optim.Adam([{"params":mlp.parameters()} for mlp in mlps], lr = LR)
loss_function=nn.CrossEntropyLoss()

for ep in range(EPOCHES):
    for img, label in trainloader:
        img,label = img.to(device), label.to(device)
        optimizer.zero_grad() #10个网络清除梯度
        for mlp in mlps:
            mlp.train()
            out = mlp(img)
            loss = loss_function(out,label)
            loss.backward()#获取梯度
        optimizer.step()
    pre = []
    vote_correct=0
    #[0,0,0]
    mlps_correct=[0 for i in range(len(mlps))]
    for img, label in testloader:
        img, label = img.to(device), label.to(device)
        for i, mlp in enumerate(mlps):
            mlp.eval()
            out=mlp(img)
            _,prediction=torch.max(out,1) #按行取最大值,索引
            pre_num=prediction.cpu().numpy()
            mlps_correct[i] += (pre_num==label.cpu().numpy()).sum()
            
            pre.append(pre_num)
        arr = np.array(pre)
        pre.clear()
        #Counter统计字符出现的次数
        result = [Counter(arr[:,i]).most_common(1)[0][0] for i in range(BATCHSIZE)]
        vote_correct += (result == label.cpu().numpy()).sum()
    print("epoch:" + str(ep) + "集成模型的正确率" + str(vote_correct/len(testloader)))
    for inx, correct in enumerate(mlps_correct):
        print("模型" + str(inx) + "正确率为：" + str(correct/len(testloader)))
            
            
        

epoch:0集成模型的正确率45.99
模型0正确率为：43.62
模型1正确率为：38.92
模型2正确率为：42.18
epoch:1集成模型的正确率51.65
模型0正确率为：46.77
模型1正确率为：44.7
模型2正确率为：49.89
epoch:2集成模型的正确率56.48
模型0正确率为：50.33
模型1正确率为：47.31
模型2正确率为：52.95
epoch:3集成模型的正确率59.3
模型0正确率为：52.14
模型1正确率为：49.47
模型2正确率为：56.17
epoch:4集成模型的正确率61.19
模型0正确率为：53.01
模型1正确率为：52.16
模型2正确率为：57.62
epoch:5集成模型的正确率61.51
模型0正确率为：54.23
模型1正确率为：53.09
模型2正确率为：58.24
epoch:6集成模型的正确率63.28
模型0正确率为：55.26
模型1正确率为：54.32
模型2正确率为：60.69
epoch:7集成模型的正确率63.62
模型0正确率为：55.96
模型1正确率为：54.89
模型2正确率为：60.85
epoch:8集成模型的正确率64.65
模型0正确率为：55.9
模型1正确率为：55.02
模型2正确率为：60.5
epoch:9集成模型的正确率65.43
模型0正确率为：56.62
模型1正确率为：56.47
模型2正确率为：61.87
epoch:10集成模型的正确率65.62
模型0正确率为：57.18
模型1正确率为：56.71
模型2正确率为：63.03
epoch:11集成模型的正确率67.16
模型0正确率为：57.75
模型1正确率为：57.68
模型2正确率为：62.2
epoch:12集成模型的正确率66.52
模型0正确率为：57.22
模型1正确率为：59.24
模型2正确率为：62.93
epoch:13集成模型的正确率67.37
模型0正确率为：57.67
模型1正确率为：59.92
模型2正确率为：63.43
epoch:14集成模型的正确率67.86
模型0正确率为：58.18
模型1正确率为：59.27
模型2正确率为：64.14
epoch:15集成模型的正确率67.86
模型0正确率为：58.51
模型1正确率为：59.03
模型2正确

In [10]:
mlps=[net4.to(device)]

optimizer=torch.optim.Adam([{"params":mlp.parameters()} for mlp in mlps],lr=LR)
  
loss_function=nn.CrossEntropyLoss()
 
for ep in range(EPOCHES):
    for img,label in trainloader:
        img,label=img.to(device),label.to(device)
        optimizer.zero_grad()#10个网络清除梯度
        for mlp in mlps:
            mlp.train()
            out=mlp(img)
            loss=loss_function(out,label)
            loss.backward()#网络们获得梯度
        optimizer.step()
 
    pre=[]
    vote_correct=0
    mlps_correct=[0 for i in range(len(mlps))]
    for img,label in testloader:
        img,label=img.to(device),label.to(device)
        for i, mlp in  enumerate( mlps):
            mlp.eval()
            out=mlp(img)
 
            _,prediction=torch.max(out,1) #按行取最大值
            pre_num=prediction.cpu().numpy()
            mlps_correct[i]+=(pre_num==label.cpu().numpy()).sum()
 
            pre.append(pre_num)
        arr=np.array(pre)
        pre.clear()
        result=[Counter(arr[:,i]).most_common(1)[0][0] for i in range(BATCHSIZE)]
        vote_correct+=(result == label.cpu().numpy()).sum()
    #print("epoch:" + str(ep)+"集成模型的正确率"+str(vote_correct/len(testloader)))
 
    for idx, coreect in enumerate( mlps_correct):
        print("VGG16模型迭代"+str(ep)+"次的正确率为："+str(coreect/len(testloader)))

VGG16模型迭代0次的正确率为：51.25
VGG16模型迭代1次的正确率为：59.4
VGG16模型迭代2次的正确率为：73.09
VGG16模型迭代3次的正确率为：78.24
VGG16模型迭代4次的正确率为：76.22
VGG16模型迭代5次的正确率为：79.81
VGG16模型迭代6次的正确率为：83.47
VGG16模型迭代7次的正确率为：80.76
VGG16模型迭代8次的正确率为：84.34
VGG16模型迭代9次的正确率为：85.4
VGG16模型迭代10次的正确率为：85.09
VGG16模型迭代11次的正确率为：86.36
VGG16模型迭代12次的正确率为：87.63
VGG16模型迭代13次的正确率为：86.61
VGG16模型迭代14次的正确率为：87.87
VGG16模型迭代15次的正确率为：88.57
VGG16模型迭代16次的正确率为：88.49
VGG16模型迭代17次的正确率为：88.89
VGG16模型迭代18次的正确率为：87.96
VGG16模型迭代19次的正确率为：88.95


TypeError: save() missing 2 required positional arguments: 'obj' and 'f'